In [ ]:
"""
Author: Matt Martin
Date: 1/27/24
Desc: Shows a few ways to generate a sequential list of integers for a dataframe/table
"""

## create the spark connection/instance
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sequence").getOrCreate()
tot_rows = 1_000_000

In [ ]:
#method 1: a loop
ints = []
for i in range(1,tot_rows+1):
    ints.append({'row_id':i})
df1 = spark.createDataFrame(ints)
df1.write.mode('overwrite').parquet('./numbers1.parquet')

In [ ]:
#method 2: built in spark range iterator
df2 = spark.range(1,tot_rows+1).toDF('row_id')
df2.write.mode('overwrite').parquet('./numbers2.parquet')

In [ ]:
spark.sql("select count(*) as row_cnt from {df}", df=df2).show()
spark.sql("select count(*) as row_cnt from {df}", df=df1).show()

In [ ]:
spark.stop